In [1]:
import xarray as xr
import xrft
import numpy as np
import dask
from tqdm import tqdm
from glob import glob
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
print(xr.__version__)
print(dask.__version__)

0.9.5-1-gab4ffee
0.14.1


In [16]:
#from dask.utils import SerializableLock
from dask.distributed import Client, wait
client = Client(scheduler_file='../../.dask_schedule_file.json')
#client.restart()
client

<Client: scheduler='tcp://10.43.8.132:8786' processes=5 cores=20>

In [7]:
client.shutdown()

In [5]:
ddir = '/rigel/ocp/users/ra2697/LLC/llc_4320_agulhas/'
#all_files = sorted(glob(ddir + '*.00000*.nc'))
all_files = sorted(glob(ddir + '*.nc'))
print(len(all_files))
print(all_files[0], all_files[-1])

1753
/rigel/ocp/users/ra2697/LLC/llc_4320_agulhas/llc_4320_agulhas.0000010368.nc /rigel/ocp/users/ra2697/LLC/llc_4320_agulhas/llc_4320_agulhas_grid.nc


In [6]:
%time ds = xr.open_mfdataset(all_files, autoclose=False, decode_cf=False)

CPU times: user 52 s, sys: 8.31 s, total: 1min
Wall time: 9min 23s


In [11]:
ds.time.attrs['units'] = 'seconds since 2011-01-01'
ds['time'] = xr.decode_cf(ds.time.to_dataset(name='foo')).time
ds.time

<xarray.DataArray 'time' (time: 1752)>
array(['2011-01-04T00:00:00.000000000', '2011-01-04T01:00:00.000000000',
       '2011-01-04T02:00:00.000000000', ..., '2011-03-17T21:00:00.000000000',
       '2011-03-17T22:00:00.000000000', '2011-03-17T23:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2011-01-04 2011-01-04T01:00:00 ...
Attributes:
    standard_name:  time
    long_name:      Time
    axis:           T

In [12]:
prefix = 'llc_4320_agulhas'
outdir = ddir + '/weekly_avg/'
outdir

'/rigel/ocp/users/ra2697/LLC/llc_4320_agulhas//weekly_avg/'

In [13]:
# should experiment more with chunk size
dsc = ds.chunk({'time': 1, 'k': 5, 'k_l': 5, 'k_u': 5, 'k_p1': 5})

In [11]:
ds_daily = dsc.resample('D', 'time')

In [9]:
# we output got the first 26
ds_daily = ds_daily.isel(time=slice(26,None))
ds_daily.time

<xarray.DataArray 'time' (time: 47)>
array(['2011-01-30T00:00:00.000000000', '2011-01-31T00:00:00.000000000',
       '2011-02-01T00:00:00.000000000', '2011-02-02T00:00:00.000000000',
       '2011-02-03T00:00:00.000000000', '2011-02-04T00:00:00.000000000',
       '2011-02-05T00:00:00.000000000', '2011-02-06T00:00:00.000000000',
       '2011-02-07T00:00:00.000000000', '2011-02-08T00:00:00.000000000',
       '2011-02-09T00:00:00.000000000', '2011-02-10T00:00:00.000000000',
       '2011-02-11T00:00:00.000000000', '2011-02-12T00:00:00.000000000',
       '2011-02-13T00:00:00.000000000', '2011-02-14T00:00:00.000000000',
       '2011-02-15T00:00:00.000000000', '2011-02-16T00:00:00.000000000',
       '2011-02-17T00:00:00.000000000', '2011-02-18T00:00:00.000000000',
       '2011-02-19T00:00:00.000000000', '2011-02-20T00:00:00.000000000',
       '2011-02-21T00:00:00.000000000', '2011-02-22T00:00:00.000000000',
       '2011-02-23T00:00:00.000000000', '2011-02-24T00:00:00.000000000',
       '2011-0

In [10]:
ds_daily.isel(time=0)['iter'].load()

<xarray.DataArray 'iter' ()>
array(101880.0)
Coordinates:
    time     datetime64[ns] 2011-01-30

In [11]:
%time daily_iternum = ds_daily['iter'].load()

CPU times: user 2.38 s, sys: 3.34 s, total: 5.72 s
Wall time: 1min 50s


In [13]:
iternum = [int(i) for i in daily_iternum]

In [14]:
keep_vars = ['U', 'V', 'W']

In [ ]:
for nt, i in enumerate(tqdm(iternum)):
    ds_day = ds_daily.isel(time=nt)
    dsets = [ds_day[v].load() for v in keep_vars]
    merged = xr.merge(dsets, )
    fname = outdir + '%s_uvw_daily.%010d.nc' % (prefix, i)
    merged.to_netcdf(fname)

 51%|█████     | 24/47 [2:50:37<5:13:52, 818.78s/it]

In [14]:
ds_weekly = dsc.resample('7D', 'time')
ds_weekly.time

<xarray.DataArray 'time' (time: 11)>
array(['2011-01-04T00:00:00.000000000', '2011-01-11T00:00:00.000000000',
       '2011-01-18T00:00:00.000000000', '2011-01-25T00:00:00.000000000',
       '2011-02-01T00:00:00.000000000', '2011-02-08T00:00:00.000000000',
       '2011-02-15T00:00:00.000000000', '2011-02-22T00:00:00.000000000',
       '2011-03-01T00:00:00.000000000', '2011-03-08T00:00:00.000000000',
       '2011-03-15T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2011-01-04 2011-01-11 2011-01-18 ...

In [15]:
%time weekly_iternum = ds_weekly['iter'].load()

CPU times: user 3.51 s, sys: 3.85 s, total: 7.35 s
Wall time: 2min 39s


In [ ]:
keep_vars = ['U', 'V', 'W']
for nt, i in enumerate(tqdm(weekly_iternum)):
    ds_week = ds_weekly.isel(time=nt)
    dsets = [ds_week[v].load() for v in keep_vars]
    merged = xr.merge(dsets, )
    fname = outdir + '%s_uvw_weekly.%010d.nc' % (prefix, i)
    merged.to_netcdf(fname)

 36%|███▋      | 4/11 [2:25:28<4:11:28, 2155.48s/it]

In [46]:
client.run(lambda dask_worker: dask_worker.worker_dir, nanny=True)

{'tcp://10.43.8.132:47460': '/local'}

In [ ]:
import env

In [ ]:
eos.environ['DASK_SCHEDULER_FILE']